In [1]:
import cv2,os, numpy as np
from PIL import Image

path = 'C:/Users/USER/UAS SISTEM CERDAS/Deteksi Masker/dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

# Function mendapatkan gambar dan label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # Mengubah ke Grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)

    return faceSamples,ids

print ("\n [Melakukan Training Wajah. Memerlukan waktu. Tunggu...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Menyimpan model pada trainer/trainer.yml
recognizer.write('trainer.yml') # recognizer.save() worked pada Mac

# Menampilkan jumlah wajah yang di training dan mengeluarkan program
print("\n [INFO] {0} Wajah Sudah di training. Keluar Program ".format(len(np.unique(ids))))


 [Melakukan Training Wajah. Memerlukan waktu. Tunggu...

 [INFO] 2 Wajah Sudah di training. Keluar Program 


In [2]:
import cv2,os,numpy as np
import winsound

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer.yml')
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");
font = cv2.FONT_HERSHEY_SIMPLEX
id = 0  #inisisasi hitungan id
names = ['none','Dengan Masker', 'Tidak Menggunakan Masker'] # nama yang berelasi dengan id : contoh ==> Dengan Masker: id=1
cam = cv2.VideoCapture(0) # Memulai Capture Video Realtime
cam.set(3, 640) # set lebar video
cam.set(4, 480) # set tinggi video
# Mendefinisikan ukuran window minimal untuk dideteksi sebagai wajah
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
while True:
    ret, img =cam.read()
    img = cv2.flip(img, 1) # Flip vertically
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale( gray, scaleFactor = 1.2, minNeighbors = 5, minSize = (int(minW), int(minH)),)
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255),2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        # Mengecek confidence lebih kecil dari 100 ==> nilai "0" cocok sempurna 
        if (confidence < 100):
            winsound.PlaySound('sirine.wav', winsound.SND_FILENAME)
            id = names[id]
        else:
            id = "Tak Dikenal"
            confidence = "  {0}%".format(round(100 - confidence))
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (0,255,255), 2)
    cv2.imshow('Deteksi Tanpa Masker',img) 
    k = cv2.waitKey(10) & 0xff # Tekan 'ESC' Untuk Menutup Aplikasi
    if k == 27:
        break
print("\n [INFO] Keluar Program")
cam.release()
cv2.destroyAllWindows()



 [INFO] Keluar Program
